Cat and Dog image classifier.

Dataset source: [https://www.microsoft.com/en-us/download/details.aspx?id=54765](https://www.microsoft.com/en-us/download/details.aspx?id=54765)

Tutorial being followed: [https://youtu.be/gT4F3HGYXf4](https://youtu.be/gT4F3HGYXf4)

Python3.8

### Part 1: Load, process, and organize the data set.

In [ ]:
# Dependencies.

!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y
!pip install opencv-python matplotlib numpy tensorflow

In [ ]:
from dataclasses import dataclass
import cv2
import numpy as np
from typing import List
import os

DATA_DIRECTORY = '/data-sets/PetImages'
CATEGORIES = ['Dog', 'Cat']
IMG_SIZE = 50

Small data structure to neatly store features and labels and keep things type safe.

In [ ]:
@dataclass
class Data:
    feature: np.ndarray
    label: int

In [14]:
def create_training_dataset(date_set_path: str, img_size: int = 100) -> List[Data]:
    """
    :param date_set_path:
    :param img_size:
    :return: List of encoded data set with labels - List[List[np.ndarray, int]]

    Loops over the 'Dog' and 'Cat' folders in the data set and returns them as a list with their labels.
    """

    encoded_data_set: List[Data] = []

    for category in CATEGORIES:
        path_to_image: str = os.path.join(date_set_path, category)
        class_num: int = CATEGORIES.index(category)

        for img in os.listdir(path_to_image):
            try:
                # Encode data and resize image.
                img_array: np.ndarray = cv2.imread(os.path.join(path_to_image, img), cv2.IMREAD_GRAYSCALE)
                resized_img: np.ndarray = cv2.resize(img_array, (img_size, img_size))

                # Image array, and it's label.
                encoded_data_set.append(Data(resized_img, class_num))

            except Exception:  # A few images are causing an issue, we are just ignoring these.
                pass

    return encoded_data_set

In [22]:
training_data: List[Data] = create_training_dataset(DATA_DIRECTORY, IMG_SIZE)

print(len(training_data))  # Check if training_date is populated.

24946


In [17]:
import random

random.shuffle(training_data)


Uppercase 'X' typically are the features, and lower case 'y' are the labels.

In [18]:
X: List[np.ndarray] = []
y: List[int] = []

for data in training_data:
    X.append(data.feature)
    y.append(data.label)

# -1 in this case means any number of images. The last number is 1 because we converted the images into gray scale.
X: np.ndarray = np.array(X).reshape((-1, IMG_SIZE, IMG_SIZE, 1))
y: np.ndarray = np.array(y)

Use pickle to store processed, and organized dataset.

In [ ]:
import pickle

with open('/pickled-data/X.pickle', 'wb') as file:
    pickle.dump(X, file)

with open('/pickled-data/y.pickle', 'wb') as file:
    pickle.dump(y, file)

### Part 2: Build and train the model.

In [11]:
# Use pickle to load the saved data.

with open('/pickled-data/X.pickle', 'rb') as file:
    X: np.ndarray = pickle.load(file)
    X = X / 255.0  # Normalize the data.

with open('/pickled-data/y.pickle', 'rb') as file:
    y: List[np.ndarray] = pickle.load(file)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from time import time

In [32]:
NAME: str = f'cat_v_dog_cnn_62X2-{int(time())}'

tensorboard = TensorBoard(
    log_dir=f'/tensorboard-logs/{NAME}'
)

# Define model architecture.
model = Sequential([
    Conv2D(64, (3, 3), input_shape=X.shape[1:], activation=relu),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation=relu),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation=relu),
    Dense(1, activation=sigmoid),
])

model.compile(
    loss=BinaryCrossentropy(),
    optimizer=Adam(),
    metrics=['accuracy']
)

In [33]:
# Train model.
history = model.fit(
    X,
    y,
    batch_size=3,
    validation_split=0.1,
    epochs=20,
    callbacks=[
        tensorboard
    ]
)

Epoch 1/20
7484/7484 [==============================] - 17s 2ms/step - loss: 0.7212 - accuracy: 0.5129 - val_loss: 0.6925 - val_accuracy: 0.5214

KeyboardInterrupt: 